In [13]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [18]:
import numpy as np
import matplotlib.pyplot as pl
import qnm_filter
import qnm
import random
import astropy.constants as c
T_MSUN = c.M_sun.value * c.G.value / c.c.value**3

In [15]:
srate = 4096*4
t_range=np.arange(-1,1,1/srate)
input = dict(model_list = [(2,2,0, 'p')],
             t_init = 0,
             segment_length = 0.2,
             srate = srate,
             ra = None, dec = None,)

In [30]:
sampling_frequency = 4096 * 1  # in Hz
duration = 8  # in second
t_range = np.arange(-duration / 2, duration / 2, 1 / sampling_frequency)

mmax = 8.4 * 1e-21
phase1 = random.uniform(0, 2 * np.pi)
A220x = mmax * np.cos(phase1)
A220y = mmax * np.sin(phase1)
phase2 = random.uniform(0, 2 * np.pi)
A221x = mmax * np.cos(phase2)
A221y = mmax * np.sin(phase2)

power220 = random.uniform(0.005, 0.3)
power221 = random.uniform(0.005, 0.3)
amp220 = np.sqrt(power220)
amp221 = np.sqrt(power221)
signal220 = np.real(amp220 * (A220x+1j*A220y) * np.exp(-1j * omega220 * np.abs(t_range / mass)))
signal221 = np.real(amp221 * (A221x + 1j * A221y) * np.exp(-1j * omega221 * np.abs(t_range / mass)))
signal = signal220 + signal221

#Loop use to be here:
bilby_ifo = qnm_filter.set_bilby_predefined_ifo(
"H1", sampling_frequency, duration, start_time=-duration / 2
)
signalH_noise = qnm_filter.bilby_get_strain(bilby_ifo, 0.0)	
signalH_no_noise = qnm_filter.RealData(signal, index=t_range, ifo="H1")
signalH = signalH_no_noise + signalH_noise

In [31]:
fit = qnm_filter.Network(segment_length=0.2, srate=4096 * 1, t_init=3.0 * mass)
fit.original_data["H1"] = signalH
fit.detector_alignment()
fit.pure_noise = {}
fit.pure_noise["H1"] = signalH_noise
fit.pure_nr = {}
fit.pure_nr["H1"] = signalH_no_noise

fit.condition_data("original_data")
fit.condition_data("pure_noise")
fit.condition_data("pure_nr")
fit.compute_acfs("pure_noise")
fit.cholesky_decomposition()
fit.first_index()

In [39]:
data = fit.truncate_data(fit.original_data)['H1']
fit.pure_nr = {}

fit.pure_nr["H1"] = qnm_filter.RealData(signal220+signal221, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template_both = fit.truncate_data(fit.pure_nr)["H1"]

fit.pure_nr["H1"] = qnm_filter.RealData(signal220, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template_220 = fit.truncate_data(fit.pure_nr)["H1"]

fit.pure_nr["H1"] = qnm_filter.RealData(signal221, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template_221 = fit.truncate_data(fit.pure_nr)["H1"]

In [44]:
import scipy.linalg as sl
template_w = sl.cho_solve((fit.cholesky_L['H1'], True), template_both)
snr_opt = np.sqrt(np.dot(template_both, template_w))

template_w = sl.cho_solve((fit.cholesky_L['H1'], True), template_220)
np.dot(data, template_w)/snr_opt

47.21609950534888

In [45]:
template_w = sl.cho_solve((fit.cholesky_L['H1'], True), template_221)
np.dot(data, template_w)/snr_opt

14.80006891320511

In [46]:
snr_opt

61.61776444850105

In [32]:
fit.pure_nr = {}
fit.pure_nr["H1"] = qnm_filter.RealData(signal220+signal221, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template = fit.truncate_data(fit.pure_nr)["H1"]
data = fit.truncate_data(fit.original_data)['H1']

SNR = fit.compute_SNR(data, template, 'H1', optimal=False)
SNR

62.0161684185541

In [35]:
fit.pure_nr = {}
fit.pure_nr["H1"] = qnm_filter.RealData(signal220, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template = fit.truncate_data(fit.pure_nr)["H1"]
data = fit.truncate_data(fit.original_data)['H1']

SNR = fit.compute_SNR(data, template, 'H1', optimal=False)
SNR

61.28558611711551

In [33]:
fit.pure_nr = {}
fit.pure_nr["H1"] = qnm_filter.RealData(signal221, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template = fit.truncate_data(fit.pure_nr)["H1"]
data = fit.truncate_data(fit.original_data)['H1']

SNR = fit.compute_SNR(data, template, 'H1', optimal=False)
SNR

58.08381210122607

In [ ]:
    def compute_SNR(self, data, template, ifo, optimal) -> float:
        """Compute matched-filter/optimal SNR.

        Parameters
        ----------
        data : ndarray
            Time-series data
        template : ndarray
            Ringdown template
        ifo : string
            Name of interferometer
        optimal: bool
            Compute optimal SNR
        """
        template_w = sl.cho_solve((self.cholesky_L[ifo], True), template)
        snr_opt = np.sqrt(np.dot(template, template_w))
        if optimal:
            return snr_opt
        else:
            snr = np.dot(data, template_w) / snr_opt
            return snr
